In [ ]:
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add
from keras.models import Model
from keras import regularizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_normal
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

In [ ]:
import os
os.chdir('drive/brat')

In [ ]:
m = keras.models.load_model('trial_MFCcascade_acc.h5')

In [ ]:
!pip3 install SimpleITK

In [ ]:
import SimpleITK as sitk
import numpy as np

In [ ]:
path = 'HG/0002'
  p = os.listdir(path)
  p.sort(key=str.lower)
  arr = []
  for i in range(len(p)):
    if(i != 4):
      p1 = os.listdir(path+'/'+p[i])
      p1.sort()
      img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[-1])
      arr.append(sitk.GetArrayFromImage(img))
    else:
      p1 = os.listdir(path+'/'+p[i])
      img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[0])
      Y_labels = sitk.GetArrayFromImage(img)    
  data = np.zeros((Y_labels.shape[1],Y_labels.shape[0],Y_labels.shape[2],4))
  for i in range(Y_labels.shape[1]):
    data[i,:,:,0] = arr[0][:,i,:]
    data[i,:,:,1] = arr[1][:,i,:]
    data[i,:,:,2] = arr[2][:,i,:]
    data[i,:,:,3] = arr[3][:,i,:]
    info = []
    for i in range(100,data.shape[0],2):
      d = data_gen(data,Y_labels,i,3)
      if(len(d) != 0):
        y = np.zeros((d[2].shape[0],1,1,5))
        for j in range(y.shape[0]):
          y[j,:,:,d[2][j]] = 1
        X1 = d[0]
        X2 = d[1]
        class_weights = class_weight.compute_class_weight('balanced',
                                                        np.unique(d[2]),
                                                        d[2])
        print('slice no:'+str(i))
        info.append(m.fit([X1,X2],y,epochs=2,batch_size=256,class_weight= class_weights))
        m.save('trial_MFCcascade_acc.h5')

In [ ]:
def model_gen(input_dim,x,y,slice_no):
  X1 = []
  X2 = []
  Y = []
  
  for i in range(int((input_dim)/2),y.shape[0]-int((input_dim)/2)):
    for j in range(int((input_dim)/2),y.shape[2]-int((input_dim)/2)):
      if(x[i-16:i+17,j-16:j+17,:].any != 0):
        X2.append(x[i-16:i+17,j-16:j+17,:])
        X1.append(x[i-int((input_dim)/2):i+int((input_dim)/2)+1,j-int((input_dim)/2):j+int((input_dim)/2)+1,:])
        Y.append(y[i,slice_no,j])
      
      
  X1 = np.asarray(X1)
  X2 = np.asarray(X2)
  Y = np.asarray(Y)
  d = [X1,X2,Y]
  return d

In [ ]:
def data_gen(data,y,slice_no,model_no):
  d = []
  x = data[slice_no]
  if(x.any() != 0 and y.any() != 0):
    if(model_no == 0):
      X1 = []
      for i in range(16,159):
        for j in range(16,199):
          if(x[i-16:i+17,j-16:j+17,:].all != 0):
            X1.append(x[i-16:i+17,j-16:j+17,:])
      Y1 = []
      for i in range(16,159):
        for j in range(16,199):
          if(x[i-16:i+17,j-16:j+17,:].all != 0):
            Y1.append(y[i,slice_no,j]) 
      X1 = np.asarray(X1)
      Y1 = np.asarray(Y1)
      d = [X1,Y1]
    elif(model_no == 1):
      d = model_gen(65,x,y,slice_no)
    elif(model_no == 2):
      d = model_gen(56,x,y,slice_no)
    elif(model_no == 3):
      d = model_gen(53,x,y,slice_no)  
    
  return d

In [ ]:
from sklearn.utils import class_weight

In [ ]:
info = []
for i in range(0,data.shape[0]):
  d = data_gen(data,Y_labels,i,3)
  if(len(d) != 0):
    y = np.zeros((d[2].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[2][j]] = 1
    X1 = d[0]
    X2 = d[1]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[2]),
                                                      d[2])
    print('slice no:'+str(i))
    info.append(m.evaluate([X1,X2],y,batch_size=256))
    #model.save('trial_0001_MFCcas_dim2_128_acc.h5')

In [ ]:
import h5py
hf = h5py.File('info_mfc.h5', 'w')
hf.create_dataset('dataset_1', data=info)
hf.close()

In [ ]:
m1 = keras.models.load_model('trial_0001_input_cascasde_acc.h5')


In [ ]:
info1 = []
for i in range(0,data.shape[0]):
  d = data_gen(data,Y_labels,i,1)
  if(len(d) != 0):
    y = np.zeros((d[2].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[2][j]] = 1
    X1 = d[0]
    X2 = d[1]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[2]),
                                                      d[2])
    print('slice no:'+str(i))
    info1.append(m1.evaluate([X1,X2],y,batch_size=256))
    #model.save('trial_0001_MFCcas_dim2_128_acc.h5')


In [ ]:
info1

In [ ]:
import h5py
hf = h5py.File('info1_input.h5', 'w')
hf.create_dataset('dataset_1', data=info1)
hf.close()

In [ ]:
info = []
for i in range(100,data.shape[0],2):
  d = data_gen(data,Y_labels,i,3)
  if(len(d) != 0):
    y = np.zeros((d[2].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[2][j]] = 1
    X1 = d[0]
    X2 = d[1]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[2]),
                                                      d[2])
    print('slice no:'+str(i))
    info.append(m.fit([X1,X2],y,epochs=2,batch_size=256,class_weight= class_weights))
    m.save('trial_MFCcascade_acc.h5')

In [ ]:
len(info)

# InputCascadeCNN training

In [ ]:
m1 = keras.models.load_model('trial_0001_input_cascasde_acc.h5')

In [ ]:
fold = os.listdir('HG/')
fold.sort(key=str.lower) 

for path in fold[4:-1]:
    print(path)
    path = 'HG/'+path
    p = os.listdir(path)
    p.sort(key=str.lower)
    arr = []
    for i in range(len(p)):
      if(i != 4):
        p1 = os.listdir(path+'/'+p[i])
        p1.sort()
        img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[-1])
        arr.append(sitk.GetArrayFromImage(img))
      else:
        p1 = os.listdir(path+'/'+p[i])
        img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[0])
        Y_labels = sitk.GetArrayFromImage(img)
    data = np.zeros((Y_labels.shape[1],Y_labels.shape[0],Y_labels.shape[2],4))
    for i in range(Y_labels.shape[1]):
      data[i,:,:,0] = arr[0][:,i,:]
      data[i,:,:,1] = arr[1][:,i,:]
      data[i,:,:,2] = arr[2][:,i,:]
      data[i,:,:,3] = arr[3][:,i,:]
    print(data.shape)
    info = []
    for i in range(154,data.shape[0],2):
      d = data_gen(data,Y_labels,i,1)
      if(len(d) != 0):
        y = np.zeros((d[2].shape[0],1,1,5))
        for j in range(y.shape[0]):
          y[j,:,:,d[2][j]] = 1
        X1 = d[0]
        X2 = d[1]
        class_weights = class_weight.compute_class_weight('balanced',
                                                          np.unique(d[2]),
                                                          d[2])
        print('slice no:'+str(i))
        info.append(m1.fit([X1,X2],y,epochs=5,batch_size=128,class_weight= class_weights))
        m1.save('trial_0001_input_cascasde_acc.h5')

In [ ]:
print(i)

# simple 2path CNN training

In [ ]:
m0 = keras.models.load_model('trial_0001_2path_acc.h5')

In [ ]:
m0.summary()

In [ ]:
info = []
for i in range(100,data.shape[0],2):
  d = data_gen(data,Y_labels,i,0)
  if(len(d) != 0):
    y = np.zeros((d[-1].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[-1][j]] = 1
    sample = np.zeros((5,1))
    for j in range(5):
      sample[j] = np.sum(y[:,:,:,j])
    print(np.transpose(sample/np.sum(sample)))    
    X1 = d[0]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[-1]),
                                                      d[-1])
    print('slice no:'+str(i))
    info.append(m0.fit(X1,y,epochs=2,batch_size=1024,class_weight= class_weights))
    m0.save('trial_0001_2path_acc.h5')

In [ ]:
import keras

In [ ]:
model = keras.models.load_model('trial_0001_MFCcas_dim2_128_acc.h5')

In [ ]:
info = []
for i in range(0,data.shape[0],2):
  d = data_gen(data,Y_labels,i,3)
  if(len(d) != 0):
    y = np.zeros((d[2].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[2][j]] = 1
    X1 = d[0]
    X2 = d[1]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[2]),
                                                      d[2])
    print('slice no:'+str(i))
    info.append(model.evaluate([X1,X2],y,batch_size=256))
    #model.save('trial_0001_MFCcas_dim2_128_acc.h5')

In [ ]:
m = keras.models.load_model('trial_0001_input_cascasde_acc.h5')

In [ ]:
info = []
for i in range(0,data.shape[0],2):
  d = data_gen(data,Y_labels,i,1)
  if(len(d) != 0):
    y = np.zeros((d[2].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[2][j]] = 1
    X1 = d[0]
    X2 = d[1]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[2]),
                                                      d[2])
    print('slice no:'+str(i))
    info.append(m.evaluate([X1,X2],y,batch_size=256))
    #model.save('trial_0001_MFCcas_dim2_128_acc.h5')

In [ ]:
hf = h5py.File('info_mfc.h5', 'r')
info = hf.get('dataset_1')

In [ ]:
info = np.asarray(info)

In [ ]:
info